In [1]:
import warnings

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import cv2
import os
import pandas as pd

2022-12-23 20:48:35.868892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 20:48:36.316060: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-23 20:48:36.316086: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-23 20:48:37.560500: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:

DATASET_PATH = r"./dataset/"
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
EPOCHS = 30
BATCH_SIZE = 32
SEED = 42
dataset_folders = []
total = 0
print(f"There are {len(os.listdir(DATASET_PATH))} folder in dataset.")
for path in sorted(os.listdir(DATASET_PATH)):
    print(f"\t-There are {len(os.listdir(DATASET_PATH + path))} images in {path} folder.")
    total += len(os.listdir(DATASET_PATH + path))
    folder_path = DATASET_PATH + path
    dataset_folders.append(folder_path)

df = pd.DataFrame(0,
                  columns=['paths',
                           'class-label'],
                  index=range(total))
i = 0
for p, path in enumerate(dataset_folders):
    for sub_path in sorted(os.listdir(path)):
        df.iloc[i, 0] = path + "/" + sub_path
        df.iloc[i, 1] = p
        i += 1
print(df.sample(frac=1, random_state=SEED).head(10))

There are 4 folder in dataset.
	-There are 300 images in 1_normal folder.
	-There are 100 images in 2_cataract folder.
	-There are 101 images in 2_glaucoma folder.
	-There are 100 images in 3_retina_disease folder.
                                         paths  class-label
110              ./dataset/1_normal/NL_111.png            0
419      ./dataset/2_glaucoma/Glaucoma_020.png            2
565  ./dataset/3_retina_disease/Retina_065.png            3
77               ./dataset/1_normal/NL_078.png            0
181              ./dataset/1_normal/NL_182.png            0
284              ./dataset/1_normal/NL_285.png            0
10               ./dataset/1_normal/NL_011.png            0
469      ./dataset/2_glaucoma/Glaucoma_070.png            2
78               ./dataset/1_normal/NL_079.png            0
349      ./dataset/2_cataract/cataract_050.png            1


In [3]:
#normalizes the histogram of images
def normalize_histograms(img):
    image_copy = img.copy()
    for i in range(3):
        imi = img[:, :, i]
        minval = np.min(imi)
        maxval = np.max(imi)
        imrange = maxval - minval
        # imi-minval will turn the color range between 0-imrange, and the scaling will stretch the range between 0-255
        image_copy[:, :, i] = (255 / (imrange + 0.0001) * (imi - minval))
    return image_copy


#Cropping image to avoid from unuseful black areas
def deleteBlackAreas(filename):
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    img = cv2.imread(filename)  #read image from file
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to grayscale
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)  # turn it into a binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # find contours
    if len(contours) != 0:
        #find the biggest area
        cnt = max(contours, key=cv2.contourArea)

        #find the bounding rect
        x, y, w, h = cv2.boundingRect(cnt)

        crop = img[y:y + h, x:x + w]  # crop image
        crop1 = cv2.cvtColor(cv2.resize(crop, image_size, interpolation=cv2.INTER_AREA),
                             cv2.COLOR_BGR2RGB)  # resize to image_size and change color space from BGR to RGB for matplotlib
        return crop1
    else:
        return cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)

In [4]:
def create_dataset(df, mode=0):
    # Creating dataset
    images = {0: [], 1: []}
    labels = {0: [], 1: []}
    limit = len(df) - len(df) // 5
    index = 0
    df = df.sample(frac=1, random_state=SEED)
    for i in range(len(df)):
        if (i + 1) >= limit:
            index = 1
        filename = df.iloc[i]["paths"]
        if i % 100 == 0:
            print(f"{i}. step---> {filename}")
        img = deleteBlackAreas(filename)
        if mode == 1:
            img = normalize_histograms(img)
        label = df.iloc[i]["class-label"]

        imlr = cv2.flip(img, 0)
        imud = cv2.flip(img, 1)

        #add all the images an labels
        images[index].append(img)
        labels[index].append(label)
        images[index].append(imlr)
        labels[index].append(label)
        images[index].append(imud)
        labels[index].append(label)

        # imb=img+0.05*img # brighter image
        # imblr=cv2.flip(imb,0)
        # imbud=cv2.flip(imb,1)
        # images.append(imb)
        # labels.append(label)
        # images.append(imblr)
        # labels.append(label)
        # images.append(imbud)
        # labels.append(label)

        # imd=img-0.075*img #deemer image
        # imdlr=cv2.flip(imd,0)
        # imdud=cv2.flip(imd,1)
        # images[index].append(imd)
        # labels[index].append(label)
        # images[index].append(imdlr)
        # labels[index].append(label)
        # images[index].append(imdud)
        # labels[index].append(label)
    print(f"{len(images[0])} train images in images dict and {len(images[1])} test images in images dict")
    print(f"{len(labels[0])} train labels in labels dict and {len(labels[1])} test labels in labels dict")
    return np.array(images[0]), np.array(images[1]), np.array(labels[0]), np.array(labels[1])


def split_dataset(X, Y):
    # set aside 20% of train and test data for evaluation
    X_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=SEED)
    # Use the same function above for the validation set
    X_train = np.array(X_train)
    x_val = np.array(x_val)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    print("X_train shape: {}".format(X_train.shape))
    print("x_val shape: {}".format(x_val.shape))
    print("y_train shape: {}".format(y_train.reshape(-1, 1).shape))
    print("y_val shape: {}".format(y_val.reshape(-1, 1).shape))
    return [X_train, x_val, y_train, y_val]

In [5]:
# Creating dataset and split the data
X_train, X_test, y_train, y_test = create_dataset(df)


0. step---> ./dataset/1_normal/NL_111.png
100. step---> ./dataset/1_normal/NL_091.png
200. step---> ./dataset/1_normal/NL_125.png
300. step---> ./dataset/1_normal/NL_195.png
400. step---> ./dataset/2_glaucoma/Glaucoma_083.png
500. step---> ./dataset/1_normal/NL_268.png



KeyboardInterrupt



In [ ]:
def plot_pie_sets(arrays):
    titles = ["Train Set", "Test Set"]
    labels = ["Normal", "Cataract", "Glaucoma", "Retina Disease"]
    fig = plt.figure(figsize=(9, 5))
    plt.title("Distribution")
    plt.axis('off')

    plt.grid(False)
    for i in range(2):
        fig.add_subplot(1, 2, i + 1)
        arr = []
        for j in range(4):
            arr.append((arrays[i] == j).sum())
        print(arr)
        plt.title(titles[i])
        plt.pie(arr)
        plt.legend(labels=labels, loc=0)


def plot_history(ar1, ar2):
    plt.plot(ar1)
    plt.plot(ar2)
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


def convertPreds(y_pred):
    res_y_pred = []
    for i in range(len(y_pred)):
        max_index = y_pred[i].argmax()
        res_y_pred.append(max_index)
    return res_y_pred


In [ ]:
plot_pie_sets([y_train, y_test])

In [ ]:
def create_model(channel_size=3):
    input_shape = []
    if channel_size == 3:  #RGB or HSV channels
        input_shape = [IMAGE_WIDTH, IMAGE_HEIGHT, channel_size]
    elif channel_size == 1:  #Grayscaled images
        input_shape = [IMAGE_WIDTH, IMAGE_HEIGHT, 1]
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(32, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.2),
        Conv2D(32, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(32, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.2),
        Flatten(),
        Dense(256, activation="relu", activity_regularizer=l1_l2()),
        Dropout(0.4),
        Dense(4, activation='softmax')
    ])
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                  metrics=['accuracy'])

    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
def strafied_kfold(model, X_train, y_train):
    # Define a StratifiedKFold object
    skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=SEED)
    acc = []
    val_acc = []
    # Iterate over the K folds
    for train_index, test_index in skf.split(X_train, y_train):
        # Split the data into K folds
        X_fold_train, X_fold_val = X_train[train_index], X_train[test_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[test_index]

        # Train and evaluate the model on the fold
        history = model.fit(X_fold_train, y_fold_train, validation_data=(X_fold_val, y_fold_val), epochs=4)
        score = model.evaluate(X_test, y_test, verbose=2)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        acc = np.concatenate((acc, history.history['accuracy']), axis=None)
        val_acc = np.concatenate((val_acc, history.history['val_accuracy']), axis=None)
    return model, acc, val_acc


In [ ]:
model, acc, val_acc = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history(acc, val_acc)
